In [ ]:
import pandas as pd
import sqlite3, plotly
import plotly.offline as py
import plotly.graph_objs as go
plotly.offline.init_notebook_mode() # run at the start of every notebook
conn = sqlite3.connect("itra.sqlite")

def getData(table):
    return pd.read_sql_query("select * from `%s`;" % table, conn)

df = getData('python')

def iplot(chart):
    py.iplot(chart)

def simpleLabel(labels, values):
    if len(labels) < 20:
        return labels, values
    total = sum(values)
    nlabels, nvalues = [], []
    other = 0
    for l, v in zip(labels, values):
        if 1.0 * v / total < 0.01:
            other += v
            continue
        nlabels.append(l)
        nvalues.append(v)
    nlabels.append('Other')
    nvalues.append(other)
    return nlabels, nvalues
    
def pieChart(title, df, column, gby=None):
    if gby is None:
        gby = column
    df = df.groupby(gby).count()['id'].reset_index()
    labels = [c if c else 'Lack of Label' for c in df[column].tolist()]
    values = df['id'].tolist()
    labels, values = simpleLabel(labels, values)
    return go.Pie(labels=labels, values=values, title=title, titlefont=dict(size=18))

iplot([pieChart('Github Trending Repo Lang Dist', df, 'lang')])

golden = getData('golden-price')
data = [go.Scatter(x=golden['_crawl_time'], y=golden['shamp'])]
layout = go.Layout(title='Golden Price Curve')
fig = go.Figure(data=data, layout=layout)
iplot(fig)

df = getData('jianshu')
iplot([pieChart('JianShu Tag Dist', df, 'tag')])

ranges = [-1,100,1000,10000,100000]
iplot([pieChart('JianShu Reads Range Percentage', df, 'reads', pd.cut(pd.to_numeric(df['reads']), ranges).astype(str))])